In [1]:
import open3d as o3d
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable

import sys
sys.path.insert(0, '../impl/utils/')
import voxel_processing as vp

batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 20

# device will determine whether to run the training on GPU or CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


device(type='cuda')

In [3]:
# Create datasets
vp.preprocess_files('../data/obj_test_data/', '../data/voxel_data')

Processing: ../data/obj_test_data/voxel_grid_1.obj -> ../data/voxel_data/voxel_grid_0
Processing: ../data/obj_test_data/voxel_grid_0.obj -> ../data/voxel_data/voxel_grid_1
2 file(s) have been processed.


In [4]:
# Load datasets
data = vp.read_data_from_path('../data/voxel_data/')
data = vp.prepare_data(data)
data

Reading: ../data/voxel_data/voxel_grid_1.obj
Reading: ../data/voxel_data/voxel_grid_0.omap
Reading: ../data/voxel_data/voxel_grid_1.offsetvec
Reading: ../data/voxel_data/voxel_grid_0.gridpoints
Reading: ../data/voxel_data/voxel_grid_0.obj
Reading: ../data/voxel_data/voxel_grid_1.omap
Reading: ../data/voxel_data/voxel_grid_0.offsetvec
Reading: ../data/voxel_data/voxel_grid_1.gridpoints
2 dataset(s) have been processed. 


[(TriangleMesh with 6130 points and 8448 triangles.,
  tensor([[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]],
  
          [[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]],
  
          [[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]],
  
          ...,
  
          [[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
  